In [28]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from joblib import dump

In [2]:
df_data = pd.read_csv('EGFR_Feature_Extraction.csv')
df_data.head()

,molecule_chembl_id,canonical_smiles,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,standard_value,class
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,41.0,active
1,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,9300.0,inactive
2,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,500000.0,inactive
3,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,3000000.0,inactive
4,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,96000.0,inactive


In [3]:
df = df_data.drop(df_data.columns[-2], axis=1, inplace=True)
df = df_data.iloc[:, 2:]
print("Null values: " + str(df.isnull().values.any()))
print(df['class'].value_counts())
df.head()

Null values: False
class
active      3810
inactive    3524
Name: count, dtype: int64


,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,class
0,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,active
1,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,inactive
2,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,inactive
3,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,inactive
4,215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,inactive


In [4]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [5]:
le = LabelEncoder()
y = le.fit_transform(y)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [79]:
model = XGBClassifier(
    n_estimators=1000, # Between 100 and 1000
    max_depth=10, # Between 3 and 10
    learning_rate=0.1, # Between 0.01 and 0.3
    gamma=0.2, # Between 0 and 0.2
    min_child_weight=5, # Between 1 and 10
    subsample=0.75, # Between 0.5 and 1
    colsample_bytree=0.75, # Between 0.5 and 1
    objective="binary:logistic", 
    booster="gbtree",   
    device="cuda",
)

eval_set = [(X_train, y_train), (X_test, y_test)]

model.fit(
    X_train,
    y_train,
    eval_set=eval_set,
    verbose=True
)

[0]	validation_0-logloss:0.66134	validation_1-logloss:0.66527
[1]	validation_0-logloss:0.63418	validation_1-logloss:0.64188
[2]	validation_0-logloss:0.60852	validation_1-logloss:0.62034
[3]	validation_0-logloss:0.58982	validation_1-logloss:0.60449
[4]	validation_0-logloss:0.56921	validation_1-logloss:0.58875
[5]	validation_0-logloss:0.55218	validation_1-logloss:0.57546
[6]	validation_0-logloss:0.53670	validation_1-logloss:0.56218
[7]	validation_0-logloss:0.52463	validation_1-logloss:0.55182
[8]	validation_0-logloss:0.51203	validation_1-logloss:0.54277
[9]	validation_0-logloss:0.50234	validation_1-logloss:0.53512
[10]	validation_0-logloss:0.49326	validation_1-logloss:0.52877
[11]	validation_0-logloss:0.48172	validation_1-logloss:0.52138
[12]	validation_0-logloss:0.47219	validation_1-logloss:0.51482
[13]	validation_0-logloss:0.46284	validation_1-logloss:0.50811
[14]	validation_0-logloss:0.45404	validation_1-logloss:0.50381
[15]	validation_0-logloss:0.44751	validation_1-logloss:0.49923
[1

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.75, device='gpu', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [80]:
y_pred = model.predict(X_test)
cr = classification_report(y_test, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       956
           1       0.84      0.81      0.82       878

    accuracy                           0.83      1834
   macro avg       0.83      0.83      0.83      1834
weighted avg       0.83      0.83      0.83      1834



In [ ]:
saved_model = "MainXGBRFClassifier.joblib"
dump(model, saved_model)